In [2]:
import os

os.environ["AZURE_APP_CONFIGURATION_URL"] = (
    "https://appcs-asesoria-des-we-01.azconfig.io"
)

In [3]:
import os

from azure.identity import DefaultAzureCredential
from pydantic import BaseModel
from pydantic_settings import BaseSettings, PydanticBaseSettingsSource
from src.pydantic_settings_azure_app_configuration.source import (
    AzureAppConfigurationSettingsSource,
)


class SqlServer(BaseModel):
    password: str
    host: str


class AzureKeyVaultSettings(BaseSettings):
    logging_level: str
    sql_server: SqlServer

    @classmethod
    def settings_customise_sources(
        cls,
        settings_cls: type[BaseSettings],
        init_settings: PydanticBaseSettingsSource,
        env_settings: PydanticBaseSettingsSource,
        dotenv_settings: PydanticBaseSettingsSource,
        file_secret_settings: PydanticBaseSettingsSource,
    ) -> tuple[PydanticBaseSettingsSource, ...]:
        azure_credential = DefaultAzureCredential()
        azure_app_configuration = AzureAppConfigurationSettingsSource(
            settings_cls,
            lambda app_configuration_options: app_configuration_options.connect_with_url(
                os.environ["AZURE_APP_CONFIGURATION_URL"], azure_credential
            )
            .select_key("my_api__*")
            .trim_key_prefix("my_api__")
            .configure_key_vault(
                lambda key_vault_options: key_vault_options.set_credential(
                    azure_credential
                )
            ),
            env_nested_delimiter="__",
        )
        return (
            init_settings,
            env_settings,
            dotenv_settings,
            file_secret_settings,
            azure_app_configuration,
        )